<span style='color:red'> NOTE: You can only pass the lab, when you provide both code and markdown </span>

Use Code for your analysis
Use Markdown to document and elaborate on your findings, conclusions, assertions, etc.

# DS_ML_I_P1: Dataset creation from raw data 
Provided is a list of Excel-Files that stem from a radar measurement using an array of 15 Antennas and a frequency sweep. Another Excel sheet provides information on the type of object that should be detected and its orientation.

The overall task is to load the data into **a single dataframe**, add the **proper information on object**, **orientation** and **the name of the image that shows the object** (which is not provided here)


## 1. Load the data and check proper loading
Load all the data into a single dataframe so that
* The name of the file is a separat column
* Only the first five columns and all rows per Sheet tab should be integrated (15 Tabs in total, one per Antenna)
* Sheet tab name should be the major index in a multiindex column dataframe
* Tab column names should be the minor index
* After this dataframe has been created the object information, orientation and image name should be added as separate columns by integrating the information from the specific excel sheet.

In [1]:
import pandas as pd
import glob
import os
import numpy as np

The code loads measurement data from multiple Excel files, each containing measurement data from 15 antennas, and organizes it into one structured DataFrame. It fulfills the task requirements by:

* Combining all files into a single DataFrame,
* Adding the filename as extra information,
* Using a MultiIndex for the columns (major: sheet name / minor: five measurement values from each sheet).

In [3]:
%%time
file_paths = glob.glob("P1b/Measurements_8_April_2023_IMP-SIMO/*.xls")

dfs =[]

for file in file_paths:
    file_data = []
    sheets = pd.read_excel(file, sheet_name=None, usecols=[0,1,2,3,4])

    for sheet_name, df in sheets.items():
        multi_columns = [
            np.repeat(sheet_name,len(df.columns)),
            df.columns.to_list()
        ]
        df.columns = pd.MultiIndex.from_arrays(multi_columns, names=['major','minor'])
        file_data.append(df)

    df["filename"] = os.path.splitext(os.path.basename(file))[0]
    dfs.append(pd.concat(file_data, axis=1))

measurement_df = pd.concat(dfs, ignore_index=True)

CPU times: total: 7.06 s
Wall time: 7.5 s


The object information was read into a DataFrame from the measurement protocol file specified in the code. Each column name was then translated into English.

In [11]:
oi = pd.read_excel("P1b/Messprotokoll_18_04_2023_open_V1.xlsx", sheet_name='Tabelle2').rename(columns={
    "Messung": "measurement", 
    "Gegenstand": "object", 
    "Postion": "position", 
    "Dateienname ": "filename", 
    "Bild ": "picture"
})

Each row's filename in the measurement dataframe (`measurement_df`) is mapped to the corresponding object, orientation, and image name in the object information DataFrame (`oi`), where the column filename was set as key. 

In [12]:
oi['filename'] = oi['filename'].astype(str).str.strip()
oi = oi.set_index('filename')

measurement_df['filename'] = measurement_df['filename'].astype(str).str.strip()

measurement_df['object'] = measurement_df['filename'].map(mp['object'])
measurement_df['orientation'] = measurement_df['filename'].map(mp['position'])
measurement_df['image_name'] = measurement_df['filename'].map(mp['picture'])

## 2. Print some statistics and analyze

## 3. Visualize the data
* Scatter Plot
* Box Plot
* Histogram

## 4. Conclusion
